# Bagging

* Temeli bootstrap yöntemi ile oluşturulan birden fazla karar ağacının
ürettiği tahminlerin bir araya getirilerek değerlendirilmesine dayanır

* veri setinden bir parça gözlem seti çekilerek bir karar ağacı oluşturulur. Bu işlem tekrarlanarak T adet karar ağacı oluşturulur
* Karar ağaçalrı oluşturma işleminde veri setinin her parçasının seçilme olasılığı eşittir. 
* ayrıca oluşturulan ağaçaların daha önceki ağaçlarla bağlantısı yoktur
* oluşturulan T adet karar ağacından gelen t adet tahmin bir araya getirilerek model tahmini oluşturulur
* Çekilen ağaçların 2/3'ü ağaçları oluşturulması için gerisi ağaçların test edilmesi içn kullanılır
* CART a göre aşırı öğrenme sorunu girderilmiştir
 

# Model

In [75]:
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split, GridSearchCV,cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale 
from sklearn import model_selection
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import BaggingRegressor

from warnings import filterwarnings
filterwarnings('ignore')

In [76]:
hit = pd.read_csv("Hitters.csv")
df = hit.copy()
df = df.dropna()
dms = pd.get_dummies(df[['League', 'Division', 'NewLeague']])
y = df["Salary"]
X_ = df.drop(['Salary', 'League', 'Division', 'NewLeague'], axis=1).astype('float64')
X = pd.concat([X_, dms[['League_N', 'Division_W', 'NewLeague_N']]], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.25, 
                                                    random_state=42)

In [77]:
bag=BaggingRegressor(bootstrap_features=True)
bag.fit(X_train,y_train)

BaggingRegressor(bootstrap_features=True)

In [78]:
bag.n_estimators#ağaç sayıusı

10

In [79]:
bag.estimators_#ağaçlar

[DecisionTreeRegressor(random_state=1147578142),
 DecisionTreeRegressor(random_state=549197831),
 DecisionTreeRegressor(random_state=810512976),
 DecisionTreeRegressor(random_state=2020617360),
 DecisionTreeRegressor(random_state=602390298),
 DecisionTreeRegressor(random_state=270134607),
 DecisionTreeRegressor(random_state=706751308),
 DecisionTreeRegressor(random_state=2091323852),
 DecisionTreeRegressor(random_state=65736456),
 DecisionTreeRegressor(random_state=401371610)]

In [80]:
bag.estimators_samples_#her ağaçtaki ornekler

[array([177,  36,  76,  66,  80,  42, 189,  32, 123,  21, 102, 124, 103,
        179, 125,  52, 179,  50, 121,  27,  10,  51, 153, 119, 164, 152,
        194, 166, 184, 113,  15,  15, 110,  79, 152, 113,  21,  49,  94,
        188, 112,  52,  11,  26,  68,   3, 165, 111,  92,   5,  67, 141,
         34, 114, 131, 187, 176, 171,  49,  72,  96,  56, 158, 166, 124,
        119, 144, 116, 119, 115,   0,  99, 184, 141,  54,  86, 184, 170,
         65, 163, 125, 120,  69, 157, 106,  29,  88, 107, 141,  64,  78,
         47,  56, 145,  69, 186, 116,  98, 130,  35, 148, 110, 109,  32,
         51, 121, 152,  85, 183,  89, 123,  29,  56,  52, 174,  89, 109,
        114, 190, 177,  28, 184,  50,  32,  64,  19,  84, 101,  79,  59,
          0, 171,  98,   0,  66, 156,  62,  58, 160,  58, 182,  69,   9,
        184,   7, 163, 171, 170, 168,  54,  31,  55, 148,  91,  39, 157,
         25, 106,  28,  47,   1,  81,  61,  81,  23,  78, 121,  27, 136,
        140, 157, 166,  41,  63, 128,   7,  98, 129

In [81]:
bag.estimators_features_#her ağaçtakş bağımsız değşkenlerin indexleri

[array([11,  9, 14,  6, 11,  0, 11, 12, 15, 16,  9, 18, 16,  4, 16,  9,  9,
        12,  3]),
 array([ 7, 12, 14, 16,  3,  3, 11, 18,  4, 18, 14,  3,  6, 18,  5, 18, 13,
         1, 16]),
 array([16,  0,  5, 11,  6,  7, 15, 15, 16, 16,  9, 12,  7, 18,  4, 13,  5,
         1, 13]),
 array([16,  2, 14, 10, 11, 17,  4, 17, 12, 17,  5, 13, 10, 13, 12, 18,  4,
        12,  1]),
 array([ 7,  4,  2, 11,  3,  3, 18,  8, 12,  5, 16, 10, 18,  7, 13,  2,  3,
         4,  9]),
 array([15, 14,  7, 13,  0,  0, 12,  2, 16,  7,  7, 13, 17,  5, 11, 10, 17,
         4,  6]),
 array([12, 11,  2,  0, 18, 14,  3, 15,  6, 12, 16,  8, 14,  2,  8, 13,  6,
        14, 13]),
 array([12,  1, 15,  2, 11, 17, 14, 15,  9, 14, 11,  8, 14, 11, 13, 17,  9,
         3,  4]),
 array([ 8,  9, 15,  4, 13, 13, 18, 13,  0,  2,  6, 15, 13, 18, 13,  1, 18,
         5,  4]),
 array([ 4,  2,  0,  1,  8, 12,  1,  3, 10, 17,  0, 12,  5,  8, 16,  2, 16,
         8, 13])]

In [82]:
bag.estimators_[0] #her bir ağaca ayrı ayroı ulaşıyoruz

DecisionTreeRegressor(random_state=1147578142)

# Tahmin

In [83]:
y_pred=bag.predict(X_test)

In [84]:
np.sqrt(mean_squared_error(y_test,y_pred))

331.0829089753083

- Şimdi ağaç ağaç değer soralım. ağaç ağaç haat değeri hesapalamay bakalım
- Buradaki asıl amaç hatalı olan hata değerini yüksek veren ağacı bulup hesap sormaktır

- Bu işlemi Şu şekilde yapıyoruz:

Her bir ağaca bag.estimators_[index] diyerek ulaşıyoruz daha sonra her bir ağaç için tekrar model kuruyoruz(bundan kastım model fit'lemek)

zaten bag ile sınıfı oluşturduk sadece her bir ağaç için fit metodunu tekrara çaalıştırıyoruz. daha sonra ise bildiiğmiz tahmin yapma

hata değeri hesaplama işlenlerini yapıyoruz

In [85]:
for i in np.arange(0,10,1):
    new_y_pred=bag.estimators_[i].fit(X_train,y_train).predict(X_test)
    print("%d.agaç = "%i,np.sqrt(mean_squared_error(y_test,new_y_pred)))

0.agaç =  524.7761597761844
1.agaç =  529.797970392231
2.agaç =  523.8755066027211
3.agaç =  463.91034911715434
4.agaç =  513.094363008729
5.agaç =  450.4445019056681
6.agaç =  519.1202163157365
7.agaç =  446.32772697810225
8.agaç =  432.4940897217644
9.agaç =  434.68541441769685
